In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [6]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [7]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [8]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [15]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [18]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [19]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [20]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [21]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(SimpleRNN(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

model.build(input_shape=(None, 200))

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 673,025 (2.57 MB)

 Trainable params: 673,025 (2.57 MB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(optimizer = "adam", loss="binary_crossentropy", metrics=["accuracy"])

In [24]:
model.fit(X_train, Y_train, epochs = 5, batch_size = 64, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 72s 139ms/step - accuracy: 0.5125 - loss: 0.7044 - val_accuracy: 0.5855 - val_loss: 0.6671
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 80s 135ms/step - accuracy: 0.6054 - loss: 0.6501 - val_accuracy: 0.6541 - val_loss: 0.6056
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 65s 129ms/step - accuracy: 0.6980 - loss: 0.5654 - val_accuracy: 0.6956 - val_loss: 0.6024
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 83s 132ms/step - accuracy: 0.7499 - loss: 0.5065 - val_accuracy: 0.7144 - val_loss: 0.5613
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 132ms/step - accuracy: 0.7536 - loss: 0.4992 - val_accuracy: 0.6273 - val_loss: 0.6385


In [25]:
model.save("model.h5")

In [26]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [27]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.6352 - loss: 0.6279


In [28]:
print(loss)

0.6316101551055908


In [29]:
print(accuracy)

0.6312999725341797


In [30]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [31]:
predictive_system("This movie was fantastic and amazing")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step


'positive'

In [32]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


'positive'

In [33]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


'positive'

In [34]:
predictive_system("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


'negative'